In [ ]:
import dualcodec
import torchaudio
from IPython.display import Audio

In [ ]:
base_id = "12hz_v2"
my_model_path = "/home/vansh/dualcodec-vansh/averaged_models/"
model_name = "averaged_model_step_0260000_decay_0.85.safetensors"

my_model = dualcodec.get_model(base_id, my_model_path, name=model_name, strict=True)
my_inference = dualcodec.InferenceWhisper(dualcodec_model=my_model)

In [ ]:
my_inference.semantic_cfg.semantic_model.encoder.causal_mask[0][0][1][:10]

In [ ]:
audio, sr = torchaudio.load("audio_samples/2400.wav")
audio = torchaudio.functional.resample(audio, sr, 24000)
audio = audio.reshape(1,1,-1)
audio = audio.cpu()
print("*")
print(f"audio.shape: {audio.shape}")
Audio(audio.squeeze(0), rate=24000)

In [ ]:
semantic_codes, acoustic_codes = my_inference.encode(audio, n_quantizers=8)

In [ ]:
print(semantic_codes.shape)
print(acoustic_codes.shape)

In [ ]:
out_audio = my_inference.decode(semantic_codes, acoustic_codes)
print("*")
Audio(out_audio.cpu().squeeze(0), rate=24000)